<a href="https://www.kaggle.com/code/sagorkumarmitra/nlp-with-transformers-library-on-huggingface?scriptVersionId=222242630" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install transformers diffusers datasets evaluate accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.1 MB/s eta 0:00:00


In [2]:
import transformers
import datasets
from datasets import load_dataset

In [3]:
reviews = load_dataset('rotten_tomatoes')

README.md:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [4]:
from huggingface_hub import notebook_login

In [5]:
notebook_login()

In [6]:
type(reviews)

datasets.dataset_dict.DatasetDict

In [7]:
reviews

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [8]:
help(reviews)

Help on DatasetDict in module datasets.dataset_dict object:

class DatasetDict(builtins.dict)
 |  A dictionary (dict of str: datasets.Dataset) with dataset transforms methods (map, filter, etc.)
 |  
 |  Method resolution order:
 |      DatasetDict
 |      builtins.dict
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __enter__(self)
 |  
 |  __exit__(self, exc_type, exc_val, exc_tb)
 |  
 |  __getitem__(self, k) -> datasets.arrow_dataset.Dataset
 |      x.__getitem__(y) <==> x[y]
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  align_labels_with_mapping(self, label2id: Dict, label_column: str) -> 'DatasetDict'
 |      Align the dataset's label ID and label name mapping to match an input `label2id` mapping.
 |      This is useful when you want to ensure that a model's predicted labels are aligned with the dataset.
 |      The alignment in done using the lowercase label names.
 |      
 |      Args:
 |          label2id (`dict`):
 |              The label name 

In [9]:
reviews['train'].to_pandas()['text'][0]

'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .'

In [10]:
from transformers import pipeline

In [11]:
classifier = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cuda:0


In [12]:
classifier("This wasn't a great movie")

[{'label': 'NEGATIVE', 'score': 0.9997251629829407}]

In [13]:
test_df = reviews['test'].to_pandas()

In [14]:
test_df['text'][0]

'lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .'

In [15]:
def score(review_text):
    return classifier(review_text)[0]['label']

In [16]:
test_df['model_prediction'] = test_df['text'].apply(score)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [17]:
test_df

,text,label,model_prediction
0,lovingly photographed in the manner of a golde...,1,POSITIVE
1,consistently clever and suspenseful .,1,POSITIVE
2,"it's like a "" big chill "" reunion of the baade...",1,NEGATIVE
3,the story gives ample opportunity for large-sc...,1,POSITIVE
4,"red dragon "" never cuts corners .",1,POSITIVE
...,...,...,...
1061,a terrible movie that some people will neverth...,0,NEGATIVE
1062,there are many definitions of 'time waster' bu...,0,NEGATIVE
1063,"as it stands , crocodile hunter has the hurrie...",0,NEGATIVE
1064,the thing looks like a made-for-home-video qui...,0,NEGATIVE


In [18]:
from transformers import AutoModelForCausalLM

In [19]:
gpt2 = AutoModelForCausalLM.from_pretrained("gpt2")

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]